# Group Convolution (Grouped Conv) — Tam Anlatım + Kod İncelemesi

Bu notebook, **Group Convolution** konusunu sıfırdan ve uygulamalı şekilde öğretmek için hazırlanmıştır.

## Bu notebook sonunda şunları net bilmiş olacaksın
1) `groups` parametresi **tam olarak neyi değiştirir**?
2) `groups=1` (normal conv) ile `groups>1` arasındaki fark
3) `groups=in_channels` neden **depthwise conv** demektir?
4) Parametre ve FLOP hesabı (kaba ama doğru sezgi)
5) PyTorch ile:
   - Group Conv katmanı
   - Depthwise + Pointwise (Depthwise Separable Conv)
   - ResNeXt tarzı **Grouped Bottleneck** blok

Not: Kodlar eğitim amaçlıdır; amaç **anlamak** ve **kendi mimarilerine entegre etmek**.


## 1) Group Convolution Nedir?

Normal konvolüsyonda:
- Her çıkış kanalı, **tüm giriş kanallarını** görür.

Group Convolution’da ise giriş kanalları `G` gruba bölünür:
- Her grup, yalnızca kendi içindeki giriş kanallarını görür.
- Çıkış kanalları da gruplara karşılık gelecek şekilde ayrılır.

### Kısa tanım
> `groups=G` demek, konvolüsyonu **G adet bağımsız Conv** gibi çalıştırmak demektir.

Bu sayede:
- Parametre azalır
- Hesap yükü azalır
- Ama kanallar arası tam karışım azalır (bunu genelde **1×1 conv** ile telafi ederiz).


## 2) PyTorch’ta `groups` Ne Demek?

PyTorch imzası:
```python
nn.Conv2d(in_channels, out_channels, kernel_size, groups=G)
```

### Geçerli olmak için şu şart gerekir:
- `in_channels % G == 0`
- `out_channels % G == 0`

Çünkü:
- Giriş kanalları G gruba bölünür → her grupta `in_channels/G` kanal olur.
- Çıkış kanalları da G gruba bölünür → her grupta `out_channels/G` kanal üretilir.


## 3) Parametre Sayısı Neden Azalıyor?

### Normal Conv (groups=1)
- Parametre: `C_out * C_in * K * K`

### Group Conv (groups=G)
Her grup bağımsız çalışır:
- Her grup için parametre: `(C_out/G) * (C_in/G) * K*K`
- Toplam grup sayısı: `G`

Toplam parametre:

\[
\frac{C_{out} C_{in} K^2}{G}
\]

Yani **yaklaşık G kat** parametre azalması.

Not: FLOP da benzer biçimde yaklaşık **G kat** azalır (aynı H×W kabulüyle).


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)


## 4) Hızlı Parametre Hesabı (Sayısal)

Aşağıdaki fonksiyonlarla normal conv ve group conv parametrelerini kıyaslayalım.


In [2]:
def params_conv2d(cin, cout, k, groups=1):
    # PyTorch Conv2d weight shape: (cout, cin/groups, k, k)
    return cout * (cin // groups) * k * k

cin, cout, k = 64, 128, 3
for G in [1, 2, 4, 8, 16, 32, 64]:
    if cin % G != 0 or cout % G != 0:
        continue
    p = params_conv2d(cin, cout, k, groups=G)
    print(f'groups={G:>2}: params={p:,}')


groups= 1: params=73,728
groups= 2: params=36,864
groups= 4: params=18,432
groups= 8: params=9,216
groups=16: params=4,608
groups=32: params=2,304
groups=64: params=1,152


## 5) En Önemli Özel Durum: `groups = in_channels` (Depthwise)

Eğer `groups = in_channels` seçersen:
- Her grup **tek bir giriş kanalına** karşılık gelir.
- Yani her kanal, kendi başına K×K filtre ile işlenir.

Bu yapı **Depthwise Convolution**’dır.

### Ama dikkat
Depthwise conv tek başına kanal karıştırmaz.
Bu yüzden genelde arkasına **1×1 conv** (Pointwise) koyarız:

```
Depthwise (spatial)  +  Pointwise 1×1 (channel mixing)
```

Bu, senin önceki çalıştığın **Depthwise Separable Conv**’un tam temelidir.


## 6) Basit Örnek: Normal Conv vs Group Conv vs Depthwise

Bu hücrede üç farklı conv oluşturacağız:
- Normal: `groups=1`
- Group: örn `groups=4`
- Depthwise: `groups=in_channels`

Ve çıktı shape’lerinin aynı olabildiğini (kanal sayıları uygunsa), ama iç işleyişin farklı olduğunu göreceğiz.


In [3]:
B, H, W = 2, 32, 32
cin, cout, k = 16, 32, 3
x = torch.randn(B, cin, H, W)

conv_normal = nn.Conv2d(cin, cout, k, padding=1, groups=1, bias=False)
conv_group  = nn.Conv2d(cin, cout, k, padding=1, groups=4, bias=False)
conv_dw     = nn.Conv2d(cin, cin,  k, padding=1, groups=cin, bias=False)  # depthwise

y1 = conv_normal(x)
y2 = conv_group(x)
y3 = conv_dw(x)

print('x :', x.shape)
print('normal:', y1.shape)
print('group :', y2.shape)
print('dw    :', y3.shape)


x : torch.Size([2, 16, 32, 32])
normal: torch.Size([2, 32, 32, 32])
group : torch.Size([2, 32, 32, 32])
dw    : torch.Size([2, 16, 32, 32])


## 7) Depthwise Separable Conv Bloğu (DW + PW)

Şimdi Depthwise (groups=in_channels) + Pointwise (1×1) bloğunu model olarak yazalım.

Bu blok, group conv’un en uç hali olan depthwise conv’un pratik kullanımını gösterir.


In [4]:
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, cin, cout, k=3, stride=1, padding=1):
        super().__init__()
        self.dw = nn.Conv2d(cin, cin, k, stride=stride, padding=padding, groups=cin, bias=False)
        self.pw = nn.Conv2d(cin, cout, 1, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.dw(x)
        x = self.pw(x)
        x = self.bn(x)
        x = self.act(x)
        return x


m = DepthwiseSeparableConv(32, 64)
inp = torch.randn(4, 32, 64, 64)
out = m(inp)
print('inp:', inp.shape, 'out:', out.shape)


inp: torch.Size([4, 32, 64, 64]) out: torch.Size([4, 64, 64, 64])


## 8) Grouped Bottleneck (ResNeXt Mantığı)

ResNeXt’in ana fikri: Bottleneck bloğunun ortasında **group conv** kullanarak
"cardinality" (paralel grup sayısı) kavramını artırmak.

Tipik bottleneck:
```
1×1 reduce → 3×3 group conv → 1×1 expand
```

Bu blok:
- Kanal sayısını 1×1 ile küçültür (reduce)
- 3×3 işlemi group conv ile daha ucuz yapar
- 1×1 ile tekrar genişletir (expand)

Çoğu zaman skip connection ile residual olur.


In [5]:
class GroupedBottleneck(nn.Module):
    def __init__(self, cin, cout, bottleneck_ratio=0.5, groups=4, stride=1):
        super().__init__()
        hidden = int(round(cout * bottleneck_ratio))

        self.reduce = nn.Sequential(
            nn.Conv2d(cin, hidden, 1, bias=False),
            nn.BatchNorm2d(hidden),
            nn.ReLU(inplace=True),
        )

        self.group_conv = nn.Sequential(
            nn.Conv2d(hidden, hidden, 3, stride=stride, padding=1, groups=groups, bias=False),
            nn.BatchNorm2d(hidden),
            nn.ReLU(inplace=True),
        )

        self.expand = nn.Sequential(
            nn.Conv2d(hidden, cout, 1, bias=False),
            nn.BatchNorm2d(cout),
        )

        if stride != 1 or cin != cout:
            self.skip = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )
        else:
            self.skip = nn.Identity()

        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = self.skip(x)
        out = self.reduce(x)
        out = self.group_conv(out)
        out = self.expand(out)
        out = out + identity
        return self.act(out)


blk = GroupedBottleneck(64, 128, groups=8, stride=2)
x = torch.randn(2, 64, 32, 32)
y = blk(x)
print('x:', x.shape, 'y:', y.shape)


x: torch.Size([2, 64, 32, 32]) y: torch.Size([2, 128, 16, 16])


## 9) Mini Model: Group Conv Kullanan CNN

Bu model:
- Stem
- 2 adet Grouped Bottleneck
- Head

Amaç: Group conv’un backbone’a nasıl oturduğunu görmek.


In [6]:
class MiniGroupNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, groups=4):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        self.stage1 = GroupedBottleneck(64, 128, groups=groups, stride=2)
        self.stage2 = GroupedBottleneck(128, 256, groups=groups, stride=2)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


model = MiniGroupNet(in_channels=3, num_classes=10, groups=8)
inp = torch.randn(4, 3, 64, 64)
out = model(inp)
print('out:', out.shape)


out: torch.Size([4, 10])


## 10) Ne Zaman Group Conv Kullanılır?

### Mantıklı olduğu yerler
- Verimlilik (FLOP/parametre) kritikse
- Backbone’da 3×3 maliyetini düşürmek istiyorsan
- ResNeXt benzeri cardinality yaklaşımı
- Depthwise separable conv tasarımları

### Dikkat / trade-off
- `groups` arttıkça kanallar arası karışım azalır.
- Bunu çoğunlukla `1×1 conv` ile telafi edersin.
- Çok yüksek `groups` bazen accuracy düşürür.

Özet:
> Group conv verimlilik sağlar; ama kanal karışımını azaltır. 1×1 conv bu dengeyi kurar.
